In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Url'].tolist()

总数量：11


['https://ecat.spectrapremium.com/en/parts/CATP04A',
 'https://ecat.spectrapremium.com/en/parts/CRP01A',
 'https://ecat.spectrapremium.com/en/parts/CRP03B',
 'https://ecat.spectrapremium.com/en/parts/FP66A',
 'https://ecat.spectrapremium.com/en/parts/GMP40B',
 'https://ecat.spectrapremium.com/en/parts/GMP66C',
 'https://ecat.spectrapremium.com/en/parts/HOP20A',
 'https://ecat.spectrapremium.com/en/parts/NSP29A',
 'https://ecat.spectrapremium.com/en/parts/NSP31A',
 'https://ecat.spectrapremium.com/en/parts/SUP06A',
 'https://ecat.spectrapremium.com/en/parts/VWP35A']

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">    <link href="/safari-pinned-tab.svg" color="#036" rel="mask-icon">    <meta content="ie=edge" http-equiv="x-ua-compatible">    <meta name="description" content="Spectra Premium believes in a green and sustainable economy. That&#039;s why it uses its core competencies of metallurgical transformation, heat transfer and high quality assembly capacity, as well as its extensive expertise, to stand out in the booming vehicle electrification sector in Quebec.">    <link href="/favicon.ico" type="image/x-icon" rel="icon"><link href="/favicon.ico" type="image/x-icon" rel="shortcut icon">    <link href="/apple-touch-icon.png" rel="apple-touch-icon">    <link href="/manifest.php?locale=en" rel="manifest">    <link href="https://fonts.googleapis.com" rel="preconnect">    <link href="https://fonts.gstatic.com" crossorigin="" rel="preconnect">    <meta name="msapplication-TileColor" content="#036">    <meta property="og:image" 

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link color="#036" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="Spectra Premium believes in a green and sustainable economy. That's why it uses its core competencies of metallurgical transformation, heat transfer and high quality assembly capacity, as well as its extensive expertise, to stand out in the booming vehicle electrification sector in Quebec." name="description"/>
  <link href="/favicon.ico" rel="icon" type="image/x-icon"/>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="/manifest.php?locale=en" rel="manifest"/>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <meta content="#036" name="msapplication-TileColor"/>
  <meta content="https://ecat.spec

In [8]:
type_ = html.xpath('//div[@class="part__canada-ligne"]/text()')[0].strip()

type_

'Oil Pan'

In [9]:
part_number = html.xpath('//h1[@class="part__titre-code"]/text()')[0].strip()

part_number

'CATP04A'

In [10]:
list_div = html.xpath('//div[@class="part__nom-spec-valeur-spec"]')

list_div

[<Element div at 0x12526d600>,
 <Element div at 0x1281bc680>,
 <Element div at 0x1282b4980>,
 <Element div at 0x1282b58c0>]

In [11]:
src = 'https://ecat.spectrapremium.com/' + html.xpath('//div[@class="part__zoom-gallery"]/descendant::img/@src')[0].strip().replace('/imagettes/', '/').replace('_220.', '.')

src

'https://ecat.spectrapremium.com//img/produits/CATP04A_ANG_P04.jpg'

In [12]:
vehicle_page = 1 if len(html.xpath('//li[@class="paginator__last"]')) == 0 else int(html.xpath('//li[@class="paginator__last"]/a/@href')[0].split('#')[0].split('=')[-1])

vehicle_page

0

In [14]:
df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                         'Type': type_,
                         'Part_Number': part_number,
                         'Vehicle': '',
                         'Pic': '',
                         'Url': input_.loc[a, 'Url'],
                         'Src': src,
                         'Vehicle_Page': vehicle_page}])

for div in list_div:
    df_temp.loc[0, div.xpath('./dt/text()')[0].strip()] = div.xpath('./dd/text()')[0].strip()

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error